## Unified version

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Parameters
Thickness = 50e-3  # Thickness of the meat in the larger point in the meat (in m)
Beta = 1.0      # Coefficient for the geometry term (set to 0 for slab, 1 for cylinder and 2 for sphere, 1.25 for cube)
T_fluid = 58.0  # Fluid temperature in °C
T_initial = 5.0  # Initial temperature of the cylinder in °C
simulation_hours = 7  # Simulation time in hours
alpha = 1.11e-7  # Thermal diffusivity in m^2/s (example for steel)
k = 0.48         # Thermal conductivity in W/(m·K) (Sanz et al., thermal conductivity of beef)
h = 95          # Convective heat transfer coefficient in W/(m^2·K)
N = 30          # Number of radial points
R = Thickness / 2  # Radius of the cylinder (10 cm diameter = 0.05 m radius)
t_max = 3600*simulation_hours # Maximum simulation time in seconds
dt = 60         # Time step in seconds

dr = R / (N - 1)  # Radial step size
r = np.linspace(0, R, N)  # Radial points from 0 to R
t = np.linspace(0, t_max, int(t_max/dt)+1)  # Time points

# Define the heat equation with convective boundary conditions
def heat_equation(t, T):
    dTdt = np.zeros_like(T)
    
    # Symmetry condition at r = 0
    #dTdt[0] = alpha * (T[1] - T[0]) / dr**2
    dTdt[0] = alpha * (2 / dr**2) * (T[1] - T[0])

    # Interior points
    for i in range(1, N - 1):
        d2T_dr2 = (T[i+1] - 2*T[i] + T[i-1]) / dr**2
        radial_term = (Beta / r[i]) * (T[i+1] - T[i-1]) / (2 * dr) if r[i] != 0 else 0
        dTdt[i] = alpha * (d2T_dr2 + radial_term)
    
    # Convective boundary condition at the outer radius
    dTdt[-1] = alpha * (1 / dr**2) * (T[-2] - T[-1] - (dr * h / k) * (T[-1] - T_fluid))
    
    return dTdt

# Solve the PDE using SciPy's solve_ivp
solution = solve_ivp(
    heat_equation,                 # ODE function
    (0, t_max),                    # Time range
    np.full(N, T_initial),         # Initial condition (uniform temperature)
    method='RK45',                 # Solver
    t_eval=t                       # Time points to evaluate
)

# Extract the solution
T_sol = solution.y
time_points = solution.t

# Plot the results
for i in range(0, len(time_points), 10):  # Plot every 20th time point
    plt.plot(r, T_sol[:, i], label=f"t = {(time_points[i])/60:.0f} m" if i <100 else None)

plt.xlabel("Radius (m)")
plt.ylabel("Temperature (°C)")
plt.title("Radial Temperature Evolution in a Heated Body")
plt.legend()
plt.grid()


In [ ]:
# Extract the temperature at the center of the cylinder (r = 0)
center_temperature = T_sol[0, :]  # First row corresponds to T[0] (r = 0)
time_points_minutes = time_points / 60  # Convert seconds to minutes

# Define 99% of the final temperature
final_temperature = T_fluid  # The final temperature is the fluid temperature
threshold_temperature = T_fluid - 0.5 #T_initial + 0.996 * (T_fluid - T_initial)   # 99% of the final temperature

# Find the first time where the center temperature exceeds the threshold
minute_reached = None
for i, temp in enumerate(center_temperature):
    if temp >= threshold_temperature:
        minute_reached = time_points_minutes[i]
        break

# Display the result
if minute_reached is not None:
    print(f"The center reaches -0.5°C of the final temperature at approximately {int(minute_reached)//60}h:{int(minute_reached)%60}m")
else:
    print("The center does not reach 99% of the final temperature within the simulated time.")

# Optional: Mark the point on the plot
plt.plot(time_points_minutes, center_temperature, label="Temperature at r = 0 (center)")
plt.axhline(threshold_temperature, color='r', linestyle='--', label="0.5°C below final temperature")
if minute_reached is not None:
    plt.axvline(minute_reached, color='g', linestyle='--', label=f"Reached at {minute_reached:.2f} min")
plt.xlabel("Time (minutes)")
plt.ylabel("Temperature (°C)")
plt.title("Temporal Variation of Temperature at the Center")
plt.legend()
plt.grid()
plt.show()

In [75]:
Dref = 2/6 # 2 minutes for 6-log reduction
Tref = 70 # Reference temperature in °C
z = 7.5 # Z-value in °C

# Initialize the cumulative total_sum array
cumulative_sum = []

# Initialize the running sum
total_sum = 0

# Loop through each temperature and calculate the contribution
for T_i in center_temperature:
    contribution = 10 ** ((T_i - Tref) / z)
    total_sum += (1/Dref) * contribution
    cumulative_sum.append(total_sum)  # Add the current total_sum to the array

# Calculate the final LR
LR = total_sum


In [4]:
from models.solvers import LogReduction
LR, cumulative_sum = LogReduction(center_temperature)

In [ ]:
# Identify the point where cumulative sum exceeds 6
threshold = 6
time_to_exceed = next((i for i, value in enumerate(cumulative_sum) if value > threshold), None)

# Plot the cumulative sum evolution
plt.figure(figsize=(8, 6))
plt.plot(time_points_minutes, cumulative_sum, marker='o', label='Cumulative Sum', color='orange')
if time_to_exceed is not None:
    plt.axvline(x=time_to_exceed + 1, color='red', linestyle='--', label=f'Reaches >6 at {(time_to_exceed + 1) // 60}h:{(time_to_exceed+1)%60}m')
plt.axhline(y=threshold, color='green', linestyle='--', label='Threshold = 6')
plt.title("Evolution of Cumulative Sum Over Time")
plt.xlabel("Time (Minutes)")
plt.ylabel("Cumulative Sum")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
None / 60

## Test packages

In [1]:
from models.parameters import MeatSimulationParameters
from models.solvers import SimulateMeat

In [2]:
msp = MeatSimulationParameters()
msp.define_meat_shape("slab",thickness_mm=5)

In [28]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from models.solvers import ExtrapolateHeatingCurve_2exp, LogReduction
from models.helpers import MeasurementFormat, seconds_to_hhmm
from models.parameters import LOG_REDUCTION_MIN_THRESHOLD

data = {
    MeasurementFormat.TIMESTAMP: [0,24, 67, 103, 120, 150],
    MeasurementFormat.TEMPERATURE: [5, 34, 54, 55, 57, 58]
}
df = pd.DataFrame(data)

In [29]:
l = ExtrapolateHeatingCurve_2exp(df,58.0)

In [33]:
# -------------------------------------------------------------------
# 4) Evaluate the fitted curve on a smooth time array
# -------------------------------------------------------------------
t_fit = np.arange(max(df[MeasurementFormat.TIMESTAMP])+120)
T_fit = l(t_fit)

In [34]:
LR_total, LR_in_time, safety_second = LogReduction(center_temperature=T_fit, dt=60)

In [35]:
safety_second

10560

In [22]:
LR_total

np.float64(10.16775367445378)

In [23]:
LR_in_time

[np.float64(6.253258929710389e-09),
 np.float64(2.0301046129616938e-08),
 np.float64(5.0627118583415216e-08),
 np.float64(1.1366122981213811e-07),
 np.float64(2.400445656820443e-07),
 np.float64(4.849075890137159e-07),
 np.float64(9.44108977541828e-07),
 np.float64(1.77898468352595e-06),
 np.float64(3.252781232272165e-06),
 np.float64(5.782526347842605e-06),
 np.float64(1.001053148252463e-05),
 np.float64(1.689992143574851e-05),
 np.float64(2.7858443477026498e-05),
 np.float64(4.48942364107269e-05),
 np.float64(7.080618584569086e-05),
 np.float64(0.0001094099490491971),
 np.float64(0.00016579874958561606),
 np.float64(0.00024663572310594056),
 np.float64(0.00036047209545840824),
 np.float64(0.0005180829828366389),
 np.float64(0.0007328103276230743),
 np.float64(0.0010209006241578852),
 np.float64(0.001401823820089633),
 np.float64(0.0018985592301207263),
 np.float64(0.0025378345405654423),
 np.float64(0.003350305021766703),
 np.float64(0.004370661844627871),
 np.float64(0.0056376608053

In [24]:
# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(t_fit, T_fit, color='blue', marker='o', label='Data Points')

# Add labels, title, and legend
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Scatter Plot of NumPy Arrays')
plt.legend()
plt.grid()

# Show the plot
plt.show()

C:\Users\montone alessio\AppData\Local\Temp\ipykernel_2460\1914829263.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
seconds_to_hhmm(9060)

'2:31'